# Welcome to the COVID-19 data insight


- The purpose of this site is to provide some additional/complementary visualization that are not typically shown by other covid-19 sites. 
- The data used in this site is from [NY Times COVID-19 data](https://github.com/nytimes/covid-19-data)
- Please send your comments and suggestions to: [jianganghao@gmail.com](jianganghao@gmail.com)


---

In [3]:
import pandas as pd
import plotly.express as px
import cufflinks as cf
from ipywidgets import interact, fixed, widgets, Layout
from plotly.subplots import make_subplots
from IPython.display import display
cf.go_offline()
cf.getThemes()
cf.set_config_file(theme='space')

In [4]:
# data from NY times github
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
dft = df.pivot_table(values='cases', index='date',columns='state',aggfunc='sum')


In [5]:
date_list = dft.index.tolist()
initial_date = widgets.SelectionSlider(options=[dt for dt in date_list], value='2020-06-01', \
                                       description='Starting', disabled=False, continuous_update=False\
                                       , orientation='horizontal', readout=True,layout=Layout(width='30%', height='40px'))
ending_date = widgets.SelectionSlider(options=[dt for dt in date_list], value='2020-06-15', \
                                       description='Ending', disabled=False, continuous_update=False\
                                       , orientation='horizontal', readout=True,layout=Layout(width='30%', height='40px'))

### The following plots display the average "acceleration" of the new cases among consecutive days
- Top panel shows the average acceleration for each state
- Bottom panel shows the epi-curve for states with positive acceleration (increase) and negative acceleration (decrease)

In [10]:
def acceleration_plot(ini_date,end_date,df):
    res=display(res=df.rolling(7).mean().loc[(df.index>=ini_date)&(df.index<= end_date),:].diff().diff().mean()\
    .iplot(kind='bar',yTitle='mean acceleration of new cases'\
           ,title='Between '+ini_date+' and '+end_date+', using 7 day rolling average',colorscale='accent',hline=0))   
    state_pos = dft.rolling(7).mean().loc[(dft.index>=ini_date)&(dft.index <= end_date),:].diff().diff().mean()[dft.loc[(dft.index>=ini_date)&(dft.index <= end_date),:].rolling(7).mean().diff().diff().mean()>=0].index.tolist()
    state_neg = dft.rolling(7).mean().loc[(dft.index>=ini_date)&(dft.index <= end_date),:].diff().diff().mean()[dft.loc[(dft.index>=ini_date)&(dft.index <= end_date),:].rolling(7).mean().diff().diff().mean()<0].index.tolist()
    df_pos = pd.DataFrame(dft.loc[:,state_pos].sum(axis=1).rolling(7).mean().diff())
    df_pos.columns = ['increase_state']
    df_neg = pd.DataFrame(dft.loc[:,state_neg].sum(axis=1).rolling(7).mean().diff())
    df_neg.columns = ['decrease_state']
    df_state = pd.merge(df_pos,df_neg,on='date')
    res=display(res=df_state.iplot(kind='bar',title='States with increase and decrease new COVID-19 cases', yTitle='Number of new cases'))
    return 
res = interact(acceleration_plot,ini_date=initial_date,end_date=ending_date,df=fixed(dft))

interactive(children=(SelectionSlider(continuous_update=False, description='Starting', index=132, layout=Layou…

### The following plot shows the epi-curve for each state

In [11]:

def epc(df,state):
    df.loc[:,state].iplot(kind='bar',yTitle='Number of new cases',title='epi-curve by state')
    return

states = df.state.unique().tolist()
states.sort()
state_list = widgets.SelectionSlider(options=[st for st in states], value='New Jersey', \
                                       description='Choose:', disabled=False, continuous_update=False\
                                       , orientation='horizontal', readout=True,layout=Layout(width='30%', height='40px'))
res = interact(epc,df=fixed(dft.rolling(7).mean().diff()),state=state_list)
    

interactive(children=(SelectionSlider(continuous_update=False, description='Choose:', index=31, layout=Layout(…